# Tribal Wildland–Urban Interface (WUI) Proximity and Pressure Index

This notebook generates a **screening-level Tribal WUI Pressure Index**
to identify where structural drivers of fast-moving wildfire
intersect with Tribal lands.

The index integrates:
- Distance to urban cores
- Road density
- Nearby population growth
- Urban expansion (impervious surface change)

This product is for **research and planning purposes only**.
Results should not be used for operational fire response or
without consultation with affected Tribal governments.

### Set up the environment
conda env create -f environment.yml
conda activate tribal-fast-fires
jupyter lab

In [7]:
# Import libraries

import os
import zipfile
import requests
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np

import osmnx as ox
import folium
import matplotlib.pyplot as plt

print("All libraries loaded")

All libraries loaded


# Datasets
- Tribal lands: Census TIGER/AIANNH
- Urban areas: Census Urban Areas
- Roads: OpenStreetMap (via osmnx)
- Population: Census (county or tract)
- Impervious surface: NLCD (placeholder here)

In [4]:
# Create data directory

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

def download_file(url, out_path):
    if out_path.exists():
        print(f"{out_path.name} already exists.")
        return
    print(f"Downloading {url}")
    r = requests.get(url, stream=True)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

In [5]:
# Download Tribal Lands (Census TIGER/AIANNH)
aiannh_zip = DATA_DIR / "aiannh.zip"
aiannh_url = "https://www2.census.gov/geo/tiger/TIGER2023/AIANNH/tl_2023_us_aiannh.zip"

download_file(aiannh_url, aiannh_zip)

with zipfile.ZipFile(aiannh_zip, "r") as z:
    z.extractall(DATA_DIR)

tribes = gpd.read_file(DATA_DIR / "tl_2023_us_aiannh.shp")
tribes = tribes.to_crs(epsg=5070)

ImportError: The 'read_file' function requires the 'pyogrio' or 'fiona' package, but neither is installed or imports correctly.
Importing pyogrio resulted in: DLL load failed while importing _io: The specified procedure could not be found.
Importing fiona resulted in: No module named 'fiona'

In [ ]:
# Download Urban Areas (Census)
urban_zip = DATA_DIR / "urban.zip"
urban_url = "https://www2.census.gov/geo/tiger/TIGER2023/UAC/tl_2023_us_uac.zip"

download_file(urban_url, urban_zip)

with zipfile.ZipFile(urban_zip, "r") as z:
    z.extractall(DATA_DIR)

urban = gpd.read_file(DATA_DIR / "tl_2023_us_uac.shp").to_crs(tribes.crs)

In [ ]:
# Distance to urban areas
urban_union = urban.geometry.unary_union

tribes["dist_to_urban_km"] = (
    tribes.geometry.centroid.distance(urban_union) / 1000
)

tribes["urban_proximity_score"] = 1 - (
    tribes["dist_to_urban_km"] / tribes["dist_to_urban_km"].max()
)

In [ ]:
# Road Density subsetted from OpenStreetMap
tribes["buffer_20km"] = tribes.geometry.buffer(20_000)

road_density = []

for geom in tribes["buffer_20km"]:
    try:
        G = ox.graph_from_polygon(
            geom, network_type="drive", simplify=True
        )
        edges = ox.graph_to_gdfs(G, nodes=False)
        road_km = edges.length.sum() / 1000
    except Exception:
        road_km = 0

    area_km2 = geom.area / 1e6
    road_density.append(road_km / area_km2)

tribes["road_density"] = road_density
tribes["road_density_score"] = (
    tribes["road_density"] / tribes["road_density"].max()
)

In [ ]:
# Population Growth (County-Level)
import cenpy

acs = cenpy.products.ACS(2022)

counties = acs.from_county(
    variables=["B01003_001E"],
    level="county"
)

counties = gpd.GeoDataFrame(
    counties,
    geometry=gpd.points_from_xy(
        counties.INTPTLON, counties.INTPTLAT
    ),
    crs="EPSG:4326"
).to_crs(tribes.crs)

counties["pop"] = counties["B01003_001E"]


In [ ]:
# Impervious Surface Change (NLCD via MRLC)
# TODO using rasterio, rasterio.mask subset by Tribal bounding box

In [ ]:
# Tribal lands (AIANNH)
tribes = gpd.read_file("data/AIANNH.shp")

# Urban areas
urban = gpd.read_file("data/urban_areas.shp")

# Ensure consistent CRS
tribes = tribes.to_crs(epsg=5070)  # CONUS Albers
urban = urban.to_crs(tribes.crs)

In [ ]:
# Calculate the minimum distance from each reservation to the nearest urban area
urban_union = urban.geometry.unary_union

tribes["dist_to_urban_km"] = (
    tribes.geometry.centroid.distance(urban_union) / 1000
)

In [ ]:
# Normalize data so that closer = higher pressure
tribes["urban_proximity_score"] = 1 - (
    tribes["dist_to_urban_km"] / tribes["dist_to_urban_km"].max()
)

# Road Density Near Reservations

### Road density is a proxy for:
- Human access
- Ignition probability
- Suppression complexity

### We calculate road length within a 20 km buffer.

In [ ]:
tribes["buffer_20km"] = tribes.geometry.buffer(20_000)

# Download OSM roads once per reservation buffer
road_densities = []

for geom in tribes["buffer_20km"]:
    try:
        G = ox.graph_from_polygon(
            geom, network_type="drive", simplify=True
        )
        edges = ox.graph_to_gdfs(G, nodes=False)
        total_km = edges.length.sum() / 1000
    except:
        total_km = 0

    area_km2 = geom.area / 1e6
    road_densities.append(total_km / area_km2)

tribes["road_density"] = road_densities

# Normalize
tribes["road_density_score"] = (
    tribes["road_density"] / tribes["road_density"].max()
)

# TODO 
5️⃣ Population Growth Nearby (Markdown + Code)

This example assumes county-level population change already joined.

# Example placeholder
tribes["pop_growth_rate"] = np.random.uniform(0, 0.3, len(tribes))

tribes["pop_growth_score"] = (
    tribes["pop_growth_rate"] / tribes["pop_growth_rate"].max()
)


In a real workflow, join Census tract or county population change
within 20 km buffers.

In [ ]:
# Urban expansion
# NLCD impervious change can be computed from raster differencing

tribes["impervious_change"] = np.random.uniform(0, 1, len(tribes))

tribes["impervious_score"] = (
    tribes["impervious_change"] / tribes["impervious_change"].max()
)

In [ ]:
# Build the Tribal WUI Pressure Index
# Weights can be adjusted collaboratively with Tribes and fire managers
weights = {
    "urban_proximity_score": 0.3,
    "road_density_score": 0.25,
    "pop_growth_score": 0.25,
    "impervious_score": 0.2
}

tribes["wui_pressure_index"] = sum(
    tribes[col] * w for col, w in weights.items()
)

In [ ]:
# Rank results
tribes["wui_rank"] = tribes["wui_pressure_index"].rank(ascending=False)

In [ ]:
# Chloropleth map
tribes.plot(
    column="wui_pressure_index",
    cmap="OrRd",
    legend=True,
    edgecolor="black"
)
plt.title("Tribal WUI Proximity & Pressure Index")
plt.axis("off")
plt.show()

In [ ]:
# Interactive map with buffers
m = folium.Map(location=[39, -98], zoom_start=4, tiles="cartodbpositron")

folium.GeoJson(
    tribes.to_crs(epsg=4326),
    style_function=lambda x: {
        "fillColor": plt.cm.OrRd(
            x["properties"]["wui_pressure_index"]
        ).hex,
        "color": "black",
        "weight": 0.5,
        "fillOpacity": 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NAME", "wui_pressure_index", "wui_rank"],
        aliases=["Tribe", "WUI Pressure", "Rank"]
    )
).add_to(m)

for dist in [5_000, 10_000, 20_000]:
    folium.GeoJson(
        tribes.to_crs(epsg=5070).geometry.buffer(dist).to_crs(4326),
        style_function=lambda x, d=dist: {
            "color": "blue",
            "weight": 1,
            "fillOpacity": 0,
        },
        name=f"{dist/1000:.0f} km buffer"
    ).add_to(m)

folium.LayerControl().add_to(m)
m

### Interpretation
High WUI Pressure scores indicate Tribal lands experiencing:
- Close proximity to expanding urban areas
- High road access (ignition risk)
- Rapid population growth nearby
- Increasing impervious surface

These patterns highlight **structural drivers of fast fires**
that Tribal governments did not create but must respond to.

### Responsible Interpretation
This index reflects **structural exposure** to fast-moving fires,
not preparedness, governance quality, or Tribal fire stewardship.

Results should be used only:
- For regional screening
- In partnership with Tribal governments
- With respect for Indigenous data sovereignty


### Next Steps
- Co-develop weights with Tribal partners
- Pair with fire occurrence and response capacity layers